In [1]:
import sys
import os
import WS
import numpy as np

Analysis code for weather station data 

This code has been used to produce the paper "Detailed analysis of local climate at the CTAO-North site from 20 yr of MAGIC weather station data"   https://dx.doi.org/10.1093/mnras/stae2214


1. Download the data from: https://dx.doi.org/10.5281/zenodo.11279074 
2. Edit and run the python script create_files.py  
3. Edit the main part of the script WS.py 
4. Run different parts of the analysis

In [2]:
# 3. Edit the main part of the script 

WS.resultdir = '../Results'    # directory in which the plots get stored  
WS.tits      = ''              # additional flag to appear in all output file names

WS.h5file_long  = '../Data/WS2003-23_long.h5'   # outcome of create_files.py
# from: https://ftp.cpc.ncep.noaa.gov/cwlinks/norm.daily.nao.cdas.z500.19500101_current.csv
WS.naoi_file = '../Data/norm.daily.nao.cdas.z500.19500101_current.csv'  # optionally, the NAOI for correlation studies
WS.not_file = '../Data/NOT_2003_2023.h5'    # optionally, if NOT data existing and needed 

WS.is_naoi = False  # set to True if naoi_file is available
WS.is_NOT  = False  # set to True if not_file is available

# Names of variables provided by the weather station
WS.name_temperature = 'temperature'
WS.name_humidity    = 'humidity'
WS.name_pressure    = 'pressure'
WS.name_ws_current  = 'windSpeedCurrent'
WS.name_ws_gust     = 'windGust'
WS.name_ws_average  = 'windSpeedAverage'
WS.name_wdir_current= 'windDirection'
WS.name_wdir_average= 'windDirectionAverage'    

# Wind binning of the Reinhardt station, see Eq. (1) of https://dx.doi.org/10.1093/mnras/stae2214
WS.wind_binning = np.concatenate((np.arange(0.,104.,4.)-0.5,105.2 + 4*(np.arange(0,10)-0.5) + 0.3*(np.arange(0,10)-0.5)**2),axis=None)

# Dates of the weather data
WS.WS_start      = '2003-01-01 00:00:01'   # Start of first data entry 
WS.WS_relocation = '2004-03-01 00:00:01'   # Relocation of the station, see Section 2.1 of https://dx.doi.org/10.1093/mnras/stae2214
WS.new_WS        = '2007-03-20 00:00:01'   # Purchase of new station MWS 5MW and upgrade 
WS.new_model     = '2017-04-10 00:00:01'   # Upgrade to a new MWS 55V 
WS.old_model     = '2023-01-16 00:00:01'   # Exchange back to repaired MWS 5MW
WS.NOT_end_of_5min = '2007-02-02 12:00:00'  # Optional, if NOT data available 
WS.NOT_end_of_data = '2019-12-31 23:59:59'  # Optional, if NOT data available

In [3]:
from setup_helper import SetUp
SetUp()

In [4]:
# Likelihood initialization, do not change unless necessary 
WS.method='L-BFGS-B'
WS.tol=1e-9

In [ ]:
WS.load_data()

In [ ]:
#plot_datacount()
#plot_downtime()
#plot_temperature()
#plot_DP()
#plot_DTR()
#plot_temperature_not()
#plot_snow()
#plot_rainy_periods()
#plot_humidity()
#plot_humidity_not()
#plot_pressure_not()
#plot_pressure()    
#plot_wind()
#plot_huracans()

